# Preparação do ambiente

## Bibliotecas

In [27]:
import json
from pathlib import Path
import flatdict
from collections import Counter
import re
from datetime import datetime
import pandas as pd
from os.path import exists, join

## Constantes e funções auxiliares

In [2]:
# datasets home folder
DATA_HOME = r'E:\Data\SCHWordCloud'

In [3]:
# number of words to extract for the word cloud
N_WORDS = 25
# pattern to match words with at least two characters
TOKEN_PATTERN = re.compile(r"\b\w\w+\b")
# version of the word cloud generator
WORD_CLOUD_VERSION = 1
# mode for the word cloud generator
WORD_CLOUD_MODE = "API"
# time format for wordcloud annotation
ANNOTATION_TS_FORMAT = "%d/%m/%Y %H:%M:%S"
# time format for json result files
RESULT_TS_FORMAT = '%Y%m%d%H%M%S%f'

In [4]:
def extract_text(items, fields_of_interest):
    flat_content = flatdict.FlatterDict(items, delimiter="__")
    target_keys = [
        key
        for key in flat_content.keys()
        for word in fields_of_interest
        if word in key
    ]
    text = " ".join([flat_content[key] for key in target_keys])
    return text

In [5]:
def extract_word_counts(text=None, n_words=N_WORDS):
    # Check if the text is empty or None
    if not text:
        return ""
    # Remove punctuation and convert to lowercase
    tokens = [
        token for token in TOKEN_PATTERN.findall(text.lower()) if token.isalpha()
    ]

    # Split the text into words and count occurrences
    words_counter = Counter(tokens)

    # Get the n most common words
    common_words = words_counter.most_common(n_words)
    wordcloud = {key: value for key, value in common_words}
    wordcloud = json.dumps(wordcloud, ensure_ascii=False)

    return wordcloud

In [6]:
def request_wordcloud_from_file(file):
    file = Path(file).absolute()
    filename = file.stem
    ts, source, query, _id = filename.split('_')
    
    with open(file, 'r') as f:
        content = json.load(f)
    
    if source == 'GOOGLE':
        fields_of_interest = ["title", "snippet", "og:title", "og:description"]
        if items := content.get('items'):
            text = extract_text(items, fields_of_interest)
        else: 
            text = ''
    elif source == 'BING':
        fields_of_interest = ["name", "snippet"]
        if webpages := content.get('webPages'):
            if items := webpages.get('value'):
                text = extract_text(items, fields_of_interest)
            else:
                text = ''
        else:
            text = ''
    
    word_counts = extract_word_counts(text)
    if word_counts == "":
        situacao = -1
    else:
        situacao = 1
    
    wordcloud_info = {
        "metaData": {
            "Version": WORD_CLOUD_VERSION,
            "Source": source,
            "Mode": WORD_CLOUD_MODE,
            "Fields": '',
            "n_words": N_WORDS,
        },
        "searchedWord": query,
        "cloudOfWords": word_counts,
    }
    wordcloud_info = json.dumps(wordcloud_info, ensure_ascii=False)
    
    wordcloud_id = _id
    wordcloud_datahora = datetime.strptime(ts,RESULT_TS_FORMAT).strftime(ANNOTATION_TS_FORMAT)
    wordcloud_computername = ""
    wordcloud_username = ""
    wordcloud_homologacao = f"{query[:5]}-{query[5:7]}-{query[7:]}"
    
    wordcloud = {
        "ID": wordcloud_id,
        "DataHora": wordcloud_datahora,
        "Computador": wordcloud_computername,
        "Usuário": wordcloud_username,
        "Homologação": wordcloud_homologacao,
        "Atributo": "WordCloud",
        "Valor": wordcloud_info,
        "Situação": situacao,
    }
    
    return wordcloud

# Desenvolvimento

## Leitura dos histórico de pesquisa

In [60]:
%%time
search_history_folder = Path(DATA_HOME) / 'search_history'
search_history_files = [file for file in search_history_folder.glob('*.json')]
search_history_parquet = Path(DATA_HOME) / 'search_history.parquet'

# save all search results history in a single parquet file
results = []
for file in search_history_files: 
    with open(file) as f:
        result = json.load(f)
        result_string = json.dumps(result)
        results.append(result_string)
df_search_history = pd.DataFrame(results,columns=['raw_contents'])
df_search_history.to_parquet(search_history_parquet)

df_search_history

CPU times: total: 15.8 s
Wall time: 2min 38s


,raw_contents
0,"{""kind"": ""customsearch#search"", ""url"": {""type""..."
1,"{""kind"": ""customsearch#search"", ""url"": {""type""..."
2,"{""kind"": ""customsearch#search"", ""url"": {""type""..."
3,"{""kind"": ""customsearch#search"", ""url"": {""type""..."
4,"{""kind"": ""customsearch#search"", ""url"": {""type""..."
...,...
46307,"{""kind"": ""customsearch#search"", ""url"": {""type""..."
46308,"{""kind"": ""customsearch#search"", ""url"": {""type""..."
46309,"{""kind"": ""customsearch#search"", ""url"": {""type""..."
46310,"{""kind"": ""customsearch#search"", ""url"": {""type""..."


In [68]:
set([s[1:].split(',')[0] for s in df_search_history.raw_contents.to_list()])

{'"_type": "SearchResponse"', '"kind": "customsearch#search"'}

## Conversão para o formato do Annotation

In [22]:
wc = [request_wordcloud_from_file(file) for file in search_history_folder.glob('*.json')]
df = pd.DataFrame(wc)
df

,ID,DataHora,Computador,Usuário,Homologação,Atributo,Valor,Situação
0,08b73c58-277c-4508-a901-4762e15da37b,23/02/2024 15:01:31,,,02018-19-01516,WordCloud,"{""metaData"": {""Version"": 1, ""Source"": ""GOOGLE""...",-1
1,8c115561-3aec-4d87-b945-af4017e706bc,23/02/2024 15:01:31,,,06618-19-01516,WordCloud,"{""metaData"": {""Version"": 1, ""Source"": ""GOOGLE""...",-1
2,91574630-05f4-4ed5-a595-422ef8b9a6d8,23/02/2024 15:01:32,,,12303-20-01516,WordCloud,"{""metaData"": {""Version"": 1, ""Source"": ""GOOGLE""...",-1
3,4c268f8d-4e7b-4c80-8e88-17cf06b015f2,23/02/2024 15:01:32,,,03744-21-13015,WordCloud,"{""metaData"": {""Version"": 1, ""Source"": ""GOOGLE""...",1
4,1dd58bdc-320b-42a8-aaaf-d117110c5777,23/02/2024 15:01:33,,,10746-20-11685,WordCloud,"{""metaData"": {""Version"": 1, ""Source"": ""GOOGLE""...",-1
...,...,...,...,...,...,...,...,...
46307,361bb851-c929-4c71-a366-af3f3ec045eb,18/04/2025 16:00:11,,,02090-17-05479,WordCloud,"{""metaData"": {""Version"": 1, ""Source"": ""GOOGLE""...",-1
46308,810e2f1f-95ac-4283-90ee-3901e1bf8062,18/04/2025 17:00:11,,,01451-14-02481,WordCloud,"{""metaData"": {""Version"": 1, ""Source"": ""GOOGLE""...",-1
46309,61843468-a47f-487b-a16e-b648fd727245,18/04/2025 17:00:11,,,01452-14-02481,WordCloud,"{""metaData"": {""Version"": 1, ""Source"": ""GOOGLE""...",-1
46310,fcd9b48c-65fa-41cc-b15f-8a3b76210f22,18/04/2025 17:00:12,,,01967-16-08748,WordCloud,"{""metaData"": {""Version"": 1, ""Source"": ""GOOGLE""...",-1


In [23]:
for w in df[df['Situação']==-1]['Valor'].sample(10):
    print(w)

{"metaData": {"Version": 1, "Source": "GOOGLE", "Mode": "API", "Fields": "", "n_words": 25}, "searchedWord": "007010803656", "cloudOfWords": ""}
{"metaData": {"Version": 1, "Source": "GOOGLE", "Mode": "API", "Fields": "", "n_words": 25}, "searchedWord": "019711401693", "cloudOfWords": ""}
{"metaData": {"Version": 1, "Source": "GOOGLE", "Mode": "API", "Fields": "", "n_words": 25}, "searchedWord": "052992214580", "cloudOfWords": ""}
{"metaData": {"Version": 1, "Source": "GOOGLE", "Mode": "API", "Fields": "", "n_words": 25}, "searchedWord": "012012301556", "cloudOfWords": ""}
{"metaData": {"Version": 1, "Source": "GOOGLE", "Mode": "API", "Fields": "", "n_words": 25}, "searchedWord": "022141203430", "cloudOfWords": ""}
{"metaData": {"Version": 1, "Source": "GOOGLE", "Mode": "API", "Fields": "", "n_words": 25}, "searchedWord": "046201803817", "cloudOfWords": ""}
{"metaData": {"Version": 1, "Source": "GOOGLE", "Mode": "API", "Fields": "", "n_words": 25}, "searchedWord": "076571912547", "clou

In [24]:
null_annotation_file = Path(DATA_HOME) / 'AnnotationNull.xlsx'
df_null = df[df['Situação']==-1]
df_null['Scarab Post Order'] = -1
df_null.to_excel(null_annotation_file,index=False)

C:\Users\maxwelfreitas\AppData\Local\Temp\ipykernel_7344\487895357.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_null['Scarab Post Order'] = -1


# Atualização do histórico de pesquisa

## Emular _cached_search_results

In [51]:
search_history_folder = Path(DATA_HOME) / 'search_history'
search_history_files = [file for file in search_history_folder.glob('*.json')]
search_history_parquet = Path(DATA_HOME) / 'search_history.parquet'

# save all search results history in a single parquet file
_cached_search_results = []
for file in search_history_files[:10]: 
    with open(file) as f:
        raw_contents = json.load(f)
        search_result = { 
            "query": "123450054321",
            "status_code": 200,
            "fields": "",
            "text": "",
            "raw_contents": raw_contents,
        }
        _cached_search_results.append(search_result)

In [56]:
_cached_search_results[0]['raw_contents']

{'kind': 'customsearch#search',
 'url': {'type': 'application/json',
  'template': 'https://www.googleapis.com/customsearch/v1?q={searchTerms}&num={count?}&start={startIndex?}&lr={language?}&safe={safe?}&cx={cx?}&sort={sort?}&filter={filter?}&gl={gl?}&cr={cr?}&googlehost={googleHost?}&c2coff={disableCnTwTranslation?}&hq={hq?}&hl={hl?}&siteSearch={siteSearch?}&siteSearchFilter={siteSearchFilter?}&exactTerms={exactTerms?}&excludeTerms={excludeTerms?}&linkSite={linkSite?}&orTerms={orTerms?}&dateRestrict={dateRestrict?}&lowRange={lowRange?}&highRange={highRange?}&searchType={searchType}&fileType={fileType?}&rights={rights?}&imgSize={imgSize?}&imgType={imgType?}&imgColorType={imgColorType?}&imgDominantColor={imgDominantColor?}&alt=json'},
 'queries': {'request': [{'title': 'Google Custom Search - 020181901516',
    'searchTerms': '020181901516',
    'count': 10,
    'startIndex': 1,
    'language': 'lang_pt',
    'inputEncoding': 'utf8',
    'outputEncoding': 'utf8',
    'safe': 'off',
    

In [59]:
%%time
search_results_data_home = r'D:\Users\maxwelfreitas\OneDrive - ANATEL\AppData\schwordcloud\datasets\search_results'
search_history_parquet = join(search_results_data_home,'search_history.parquet')

# just for development, in real cenario get raw_contents from search_results
search_result_raw_contents = [json.dumps(result['raw_contents']) for result in _cached_search_results]
df_search_results = pd.DataFrame(search_result_raw_contents,columns=['search_history'])

if exists(search_results_data_home):
    df_search_history = pd.read_parquet(search_history_parquet)
    df_search_results = pd.concat([df_search_history,df_search_results])

if df_search_results.empty:
    return

try:  
    df_search_results.to_parquet(search_history_parquet)
    _cached_search_results = []
    

CPU times: total: 0 ns
Wall time: 869 μs


,search_history
0,"{""kind"": ""customsearch#search"", ""url"": {""type""..."
1,"{""kind"": ""customsearch#search"", ""url"": {""type""..."
2,"{""kind"": ""customsearch#search"", ""url"": {""type""..."
3,"{""kind"": ""customsearch#search"", ""url"": {""type""..."
4,"{""kind"": ""customsearch#search"", ""url"": {""type""..."
5,"{""kind"": ""customsearch#search"", ""url"": {""type""..."
6,"{""kind"": ""customsearch#search"", ""url"": {""type""..."
7,"{""kind"": ""customsearch#search"", ""url"": {""type""..."
8,"{""kind"": ""customsearch#search"", ""url"": {""type""..."
9,"{""kind"": ""customsearch#search"", ""url"": {""type""..."


In [46]:
results[0]

'{"kind": "customsearch#search", "url": {"type": "application/json", "template": "https://www.googleapis.com/customsearch/v1?q={searchTerms}&num={count?}&start={startIndex?}&lr={language?}&safe={safe?}&cx={cx?}&sort={sort?}&filter={filter?}&gl={gl?}&cr={cr?}&googlehost={googleHost?}&c2coff={disableCnTwTranslation?}&hq={hq?}&hl={hl?}&siteSearch={siteSearch?}&siteSearchFilter={siteSearchFilter?}&exactTerms={exactTerms?}&excludeTerms={excludeTerms?}&linkSite={linkSite?}&orTerms={orTerms?}&dateRestrict={dateRestrict?}&lowRange={lowRange?}&highRange={highRange?}&searchType={searchType}&fileType={fileType?}&rights={rights?}&imgSize={imgSize?}&imgType={imgType?}&imgColorType={imgColorType?}&imgDominantColor={imgDominantColor?}&alt=json"}, "queries": {"request": [{"title": "Google Custom Search - 020181901516", "searchTerms": "020181901516", "count": 10, "startIndex": 1, "language": "lang_pt", "inputEncoding": "utf8", "outputEncoding": "utf8", "safe": "off", "cx": "e4d9c4b82b2a44914", "cr": "c